In [2]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf


In [3]:
path = os.getcwd()
parent = os.path.dirname(path)
os.chdir(parent + '/dataset_cleaned/')


In [4]:
monthly_generation = pd.read_csv('monthly_generation.csv')

monthly_generation = monthly_generation[['Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Month',
                                         'Generation']]

monthly_generation['Generation'] = monthly_generation['Generation'] * 1000

monthly_generation['Month'] = pd.Categorical(monthly_generation['Month'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                                                                           'Oct', 'Nov', 'Dec'])
monthly_generation = monthly_generation.sort_values(
    by=['Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Month'])


In [5]:
monthly_generation


,Year,State,Plant Code,Aggregated Fuel Group,Month,Generation
232,2013,AK,75,GAS,Jan,2229680.0
231,2013,AK,75,GAS,Feb,1819216.0
235,2013,AK,75,GAS,Mar,1979419.0
228,2013,AK,75,GAS,Apr,1750900.0
236,2013,AK,75,GAS,May,1790274.0
...,...,...,...,...,...,...
241525,2020,WY,60130,GAS,Aug,2144000.0
241535,2020,WY,60130,GAS,Sep,2091000.0
241534,2020,WY,60130,GAS,Oct,2145000.0
241533,2020,WY,60130,GAS,Nov,2146000.0


### Calculate the coal percentage matrix


In [6]:
generation_coal = pd.read_csv('coal_monthly_generation_by_state.csv')
generation_gas = pd.read_csv('gas_monthly_generation_by_state.csv')


In [7]:
generation_coal.insert(3, 'Aggregated Fuel Group', 'COAL')
generation_gas.insert(3, 'Aggregated Fuel Group', 'GAS')


In [8]:
generation = pd.concat([generation_coal, generation_gas])


In [9]:
# calculate month / year percentage for each state

a = pd.pivot_table(generation, index=['Year', 'State', 'Aggregated Fuel Group'], columns=[
                   'Month'], values='Generation', dropna=False)
a = a[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

b = generation.groupby(['Year', 'State', 'Aggregated Fuel Group']).aggregate(
    {'Generation': 'sum'})

generation_monthly_percentage = np.divide(a, b)
generation_monthly_percentage.head()


/var/folders/xy/8mx9wl7x3h19xrmkncxcmw3r0000gn/T/ipykernel_19733/1267620063.py:11: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  generation_monthly_percentage = np.divide(a, b)


Month                                  Jan       Feb       Mar       Apr  \
Year State Aggregated Fuel Group                                           
2013 AK    COAL                   0.094504  0.086242  0.090347  0.063786   
           GAS                    0.100952  0.082260  0.089421  0.079059   
     AL    COAL                   0.075630  0.067621  0.076704  0.078566   
           GAS                    0.080510  0.093623  0.095553  0.067512   
     AR    COAL                   0.085228  0.076039  0.077372  0.073093   

Month                                  May       Jun       Jul       Aug  \
Year State Aggregated Fuel Group                                           
2013 AK    COAL                   0.059490  0.084464  0.085011  0.086924   
           GAS                    0.080734  0.070892  0.081175  0.077235   
     AL    COAL                   0.084081  0.091643  0.090542  0.099708   
           GAS                    0.064138  0.086802  0.090411  0.097972   
     AR    COAL                   0.075644  0.089857  0.100570  0.102301   

Month                                  Sep       Oct       Nov       Dec  
Year State Aggregated Fuel Group                                          
2013 AK    COAL                   0.086759  0.087629  0.088607  0.086237  
           GAS                    0.081196  0.070316  0.084074  0.102686  
     AL    COAL                   0.093902  0.085688  0.076846  0.079069  
           GAS                    0.079333  0.080685  0.081721  0.081740  
     AR    COAL                   0.091274  0.070247  0.063182  0.095193

In [10]:
yearly_generation_emission = pd.read_csv('yearly_generation_emission.csv').set_index(
    ['Year', 'State', 'Aggregated Fuel Group', 'Plant Code'])


In [11]:
yearly_generation_emission

Generation      Emission
Year State Aggregated Fuel Group Plant Code                            
2013 AK    GAS                   75          2.216302e+07  2.077440e+04
           COAL                  79          1.878430e+08  3.511266e+05
           GAS                   96          1.379744e+09  8.745762e+05
           COAL                  6288        1.910638e+08  2.658208e+05
           GAS                   6292        4.196500e+07  3.956033e+04
...                                                   ...           ...
2020 WY    COAL                  56609       3.203757e+09  3.571382e+06
           GAS                   57703       3.542980e+08  1.624272e+05
           COAL                  57915       1.468371e+08  9.347161e+05
           GAS                   57915       6.580391e+07  2.261733e+05
                                 60130       2.725400e+07  1.661524e+04

[20597 rows x 2 columns]

In [12]:
# calculate the monthly generation according to the state's monthly generation percentage

data = []
for p in yearly_generation_emission.index:

    year, state, fuel_group, plant = p[0], p[1], p[2], p[3]

    generation = yearly_generation_emission.loc[p]['Generation']

    percentage = generation_monthly_percentage.loc[(year, state, fuel_group)]

    data.append([year, state, plant,  fuel_group] +
                list(np.array(generation) * np.array(percentage)))

monthly = pd.DataFrame(data, columns=['Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
                                      'Sep', 'Oct', 'Nov', 'Dec'])

monthly = pd.melt(monthly, id_vars=['Year', 'State', 'Plant Code', 'Aggregated Fuel Group']).rename(
    {'variable': 'Month', 'value': 'Generation'}, axis=1)

monthly['Month'] = pd.Categorical(monthly['Month'], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
                                                     'Oct', 'Nov', 'Dec'])
monthly = monthly.sort_values(
    by=['Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Month'])
monthly = monthly.reset_index(drop=True)

monthly_generation_predict = monthly


In [13]:
# combine
merged = pd.merge(monthly_generation, monthly_generation_predict,  how='right', left_on=[
                  'Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Month'], right_on=['Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Month'])

merged['Generation'] = merged['Generation_x'].combine_first(
    merged['Generation_y'])

del merged['Generation_x']
del merged['Generation_y']

monthly_generation_merged = merged


In [14]:
yearly_generation_emission.reset_index(inplace=True)

In [15]:
# get the generation to emission function

gas_yearly_generation_emission = yearly_generation_emission[
    yearly_generation_emission['Aggregated Fuel Group'] == 'GAS']

model_gas = smf.ols(formula='Emission ~ Generation',
                    data=gas_yearly_generation_emission)
model_gas = model_gas.fit()
model_gas.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Emission   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                 5.740e+05
Date:                Tue, 25 Oct 2022   Prob (F-statistic):               0.00
Time:                        12:01:15   Log-Likelihood:            -2.2351e+05
No. Observations:               17031   AIC:                         4.470e+05
Df Residuals:                   17029   BIC:                         4.470e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.858e+04    999.434     38.604      0.000    3.66e+04    4.05e+04
Generation     0.0004   5.75e-07    757.603      0.000       0.000       0.000
==============================================================================
Omnibus:                    20291.601   Durbin-Watson:                   1.605
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7003276.895
Skew:                           5.950   Prob(JB):                         0.00
Kurtosis:                     101.628   Cond. No.                     1.87e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.87e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# get the generation to emission function

coal_yearly_generation_emission = yearly_generation_emission[
    yearly_generation_emission['Aggregated Fuel Group'] == 'COAL']

model_coal = smf.ols(formula='Emission ~ Generation',
                     data=coal_yearly_generation_emission)
model_coal = model_coal.fit()
model_coal.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Emission   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 4.207e+05
Date:                Tue, 25 Oct 2022   Prob (F-statistic):               0.00
Time:                        12:01:17   Log-Likelihood:                -50722.
No. Observations:                3566   AIC:                         1.014e+05
Df Residuals:                    3564   BIC:                         1.015e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.649e+05   7599.864     21.698      0.000     1.5e+05     1.8e+05
Generation     0.0011   1.64e-06    648.622      0.000       0.001       0.001
==============================================================================
Omnibus:                     1874.382   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37282.769
Skew:                           2.052   Prob(JB):                         0.00
Kurtosis:                      18.300   Cond. No.                     5.76e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
coal_monthly_generation = monthly_generation_merged[monthly_generation_merged['Aggregated Fuel Group'] == 'COAL']
coal_monthly_generation['Emission (Predicted)'] = model_coal.predict(coal_monthly_generation)

gas_monthly_generation = monthly_generation_merged[monthly_generation_merged['Aggregated Fuel Group'] == 'GAS']
gas_monthly_generation['Emission (Predicted)'] = model_gas.predict(gas_monthly_generation)

/var/folders/xy/8mx9wl7x3h19xrmkncxcmw3r0000gn/T/ipykernel_19733/230044131.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coal_monthly_generation['Emission (Predicted)'] = model_coal.predict(coal_monthly_generation)
/var/folders/xy/8mx9wl7x3h19xrmkncxcmw3r0000gn/T/ipykernel_19733/230044131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gas_monthly_generation['Emission (Predicted)'] = model_gas.predict(gas_monthly_generation)


In [18]:
monthly_generation_emission = pd.concat([coal_monthly_generation, gas_monthly_generation])

monthly_generation_emission = monthly_generation_emission.sort_values(
    by=['Year', 'State', 'Plant Code', 'Aggregated Fuel Group', 'Month'])

In [19]:
monthly_generation_emission.to_csv('monthly_generation_emission.csv', index=False)